# Analyzing borrowers’ risk of defaulting

Our project is to prepare a report for a bank’s loan division. I’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank has some data on customers’ credit worthiness.

The report will be considered when building a credit scoring of a potential customer, which is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer

In [2]:
try:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')
except:
    df = pd.read_csv('/credit_scoring_eng.csv')

In [3]:
# Checking general DF info
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [4]:
# Checking general DF info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

The column days_employed seems to have a lot of impossible numbers, mostly consisting of negative days and days that go past the life-span of the one working in it - for now it seems that all of those with an extreme amount of days employed are retired, and are likely an error, while those in the negative are reasonable and likely correct and simply need to be turned into positive numbers.

## Data preprocessing

### Processing missing values

In [5]:
df['days_employed'] = df['days_employed'].abs()
df.loc[df['income_type'].eq('retiree'), 'days_employed'] = np.nan

### Conclusion

Some rows have NaN as both days_employed and total_income for an individual. There may be the possibility that it's because they no longer have the job anymore and may be unemployed, but some with NaN values for both are listed as employees under income_type. It could be possible they lied about being employeed and continued filling the rest of the questionare with N/A or something similar for income-related questions. These have an undetermined reason for why they are NaN but aren't completely random, so they will be kept in the data and left as they are, since I can still work with these rows as they are.

### Data type replacement

In [6]:
# Using .contains() to replace strings with more general
df.loc[df['purpose'].str.contains('car', 'purpose'), 'purpose'] = 'car'
df.loc[df['purpose'].str.contains('educat|university'), 'purpose'] = 'education'
df.loc[df['purpose'].str.contains('wedding'), 'purpose'] = 'marriage'
df.loc[df['purpose'].str.contains('hous|estate|propert'), 'purpose'] = 'house'

df['dob_years'] = df['dob_years'].replace([0], np.NaN)
df['children'] = df['children'].replace([-1], np.NaN)

In [21]:
# Ensuring there are minimal responses
df['purpose'].unique()

array(['house', 'car', 'education', 'marriage'], dtype=object)

In [22]:
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [23]:
df['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

### Conclusion

In the purpose section of the loan reason there was significant variation in what is basically only four main reasons for the purpose of each loan. For some the reasons go outside the assigned category slightly, such as changing the purpose of building or renovating a house to simply a housing purchase. These variations are mostly insignificant and narrowing the categories will help the data be much more easy to work with.

I also fixed the errors in the data where it was clear it was an impossible number, such as having negative one children. There wasn't reason to believe it may have meant 0 children, or even 1, so I instead made it NaN. While it is true that there was a negative amount of days when it simply needed to be postive to be correct, there's too little here to prove that it's the same case and removing these values won't affect the data significantly.

### Processing duplicates

In [27]:
df['family_status'] = df['family_status'].str.lower()
df['education'] = df['education'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

df = df.drop_duplicates().reset_index(drop=True)

### Conclusion

There are some capitals and lowercase letters in both the education and purpose column that may make finding duplicates harder, so I converted all of these strings to lowercase. After simplifying the purpose reasons into four it categories drops much more rows and turning each column with string values into lowercase, drop_duplicates() droped a noticable amount of rows. In the education column there was three different strings for what was basically just one. For example, for 'some college', there was also a version with just the first letter capitalized (Some college) and also a version with it all capitalized (SOME COLLEGE). This was the same for the other actually unique values in education. There may be an issue with how the data was processed that caused it to create two new categories for each three possible inputs for the same category.

### Categorizing Data

In [44]:
# Total clients
total_clients = len(df)

# Correlation between having children and having debt
pvt1 = df.pivot_table(index='children', values='debt', aggfunc=np.size)

def children_row(row):
    if row['children'] >= 1 :
        return 1
    else:
        return 0

df['children_or_not'] = df.apply(lambda row: children_row(row), axis=1)
pvt1 = pvt1.drop([4, 5, 20])

debt_0_children = 0
debt_1_children = 0
debt_2_children = 0
debt_3_children = 0
for index, row in df.iterrows():
    if row['debt'] == 1 and row['children'] == 0:
        debt_0_children += 1
    if row['debt'] == 1 and row['children'] == 1:
        debt_1_children += 1
    if row['debt'] == 1 and row['children'] == 2:
        debt_2_children += 1
    if row['debt'] == 1 and row['children'] == 3:
        debt_3_children += 1

pvt1['with_debt'] = 'x'
pvt1.at[0, 'with_debt']=debt_0_children
pvt1.at[1, 'with_debt']=debt_1_children
pvt1.at[2, 'with_debt']=debt_2_children
pvt1.at[3, 'with_debt']=debt_3_children

def conversion(row):
    return row['with_debt'] / row['debt']

pvt1['probability'] = pvt1.apply(lambda row: conversion(row), axis=1)

# Correlation between being family status and having debt
family_married = len(df.loc[df['family_status'] == 'married']) / total_clients
family_civil = len(df.loc[df['family_status'] == 'civil partnership']) / total_clients
family_widowed = len(df.loc[df['family_status'] == 'widow / widower']) / total_clients
family_divorced = len(df.loc[df['family_status'] == 'divorced']) / total_clients
family_unmarried = len(df.loc[df['family_status'] == 'unmarried']) / total_clients

# Correlation between income levels and having debt
income_section_1 = np.percentile(df['total_income'], 33)
income_section_2 = np.percentile(df['total_income'], 66)

under_class = len(df.loc[df['total_income'] >= income_section_2]) / total_clients
middle_class = len(df.loc[df['total_income'].between(income_section_1, income_section_2)]) / total_clients
upper_class = len(df.loc[df['total_income'] <= income_section_1]) / total_clients

# Correlation between the purpose of the loan and the rate of debt
purpose_housing = len(df.loc[df['purpose'] == 'house']) / total_clients
purpose_education = len(df.loc[df['purpose'] == 'education']) / total_clients
purpose_cars = len(df.loc[df['purpose'] == 'car']) / total_clients
purpose_marriage = len(df.loc[df['purpose'] == 'marriage']) / total_clients

In [46]:
np.percentile(df, 66)

## abandoned here

nan

### Conclusion

Here I made sections of code for each question to set the values and set up how I plan on answering each of the questions for the credit check. Each risk check will be a conversion to see which category is more likely to be a risk of late payment on a loan, except for kids and debt rate, which will be a pivot table. For total income and debt, I used the median of all incomes given to find the break point for what will be considered those with a higher income level and those with a lower level. I chose the median as it will prevent the baseline from being affected too heavily by abnormally high or low income levels. This will give a general idea of if higher income will help or hurt when deciding credit for a loan.

When attempting to use a pivot table to answer the relation between having children and being able to repay a loan, I dropped the rows for those with children of 4, 5, and 20. The reason being that there are too little loaners with those amount of children to make any good conclusion out of, compared to the rest of the data. If I were to allowed those three, it may skew the conclusion as since there are fewer values it likely would vary significantly where it otherwise shouldn't.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time? 

Yes, using the table pvt1 we can see how having kids increase the chance of defaulting on a loan. While only increasing about 2% in total from having 0 children to 1, this is close to about a 25% increase in rate compared to the jump from 7% to 9%. The rate increases slightly for those that have 2 kids and then drops back down to 8% for those that have 3 kids, which is still a noticably higher rate than those without. It is likely the case that having to pay for children would make paying back a loan harder as the kids themselves take money away from what could have been a loan repayment.

In [10]:
pvt1

,debt,with_debt,probability
children,,,
0.0,14149,1063,0.075129
1.0,4818,444,0.092154
2.0,2055,194,0.094404
3.0,330,27,0.081818


<div class="alert alert-danger"> <b>Reviewer comment:</b> 
    
<s>Please note that we did not count correctly to answer the questions.
    
Here we need to calculate the client's probability of becoming a debtor.
    
How is it calculated?
    
Probability = number of clients who did not repay debt / total number of clients in the group
    
This can be calculated using pivot tables `pivot_table`, I want to remind you that pivot tables are the same dataframes. That is, you can add columns to them and do the calculations we need.
    
Let's correct these calculations, please.
    
    
---
    
    
We also need to create some categories so that we can use pivot tables to answer questions.
    
</div>

<div class="alert alert-info"> If the formula is debtors / total clients, wouldn't it be easiest to edit my existing code and simply change the bottom number to be divided in my ratio variables to the total clients, instead of setting up a pivot table?</div>

<div class="alert alert-success"> <b>Reviewer comment (2):</b> 


The point is that pivot tables are a very useful tool in our practice. Each student has a different level of training and learning speed, so it is important for us to highlight such moments that are especially important in our practice.

Let's fix this point for one question so that I can make sure that we have dealt with this question.
    
</div>

<div class="alert alert-info"> I wasn't entirely sure how you wanted me to go about doing it using a pivot table, so I went about it the best way I could think of. For the point I used it on I just did the first question, which was having children and the rate of debt. It definitely didn't feel easier than my other way of doing it, but I also feel that I probably made it harder than I needed to. If you can suggest ways of improving my attempt it'd be great to take note of.
    

I also interpreted "create some categories" as making a 0 and 1 column for each loaner for 0 if they had no children and 1 if they did, which became df['children_or_not'], although I had trouble trying to figure out how to incorporate that into my probability column as it seemed better to instead find the probability based off each amount of children (0-3) rather than simply whether they had kids or not, which the column would help with.
</div>

<div class="alert alert-success"> <b>Reviewer comment (2):</b> 
    
An interesting way to calculate.

I had to write to me that there were difficulties with the calculation, I would try to look at abstract examples to explain.
    
I left an example below how you can simplify the calculation itself.
    
</div>

In [11]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_or_not
0,1.0,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,house,1
1,1.0,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,1
2,0.0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,house,0
3,3.0,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,1
4,0.0,NaN,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,marriage,0


In [12]:
report_children_factors = df.pivot_table(
    index="children",
    values="debt",
    aggfunc=["sum", "count"]
)

In [13]:
report_children_factors

,sum,count
,debt,debt
children,,
0.0,1063,14149
1.0,444,4818
2.0,194,2055
3.0,27,330
4.0,4,41
5.0,0,9
20.0,8,76


In [14]:
# drop excess level of columns 

report_children_factors.columns = report_children_factors.columns.droplevel(1)

In [15]:
report_children_factors

,sum,count
children,,
0.0,1063,14149
1.0,444,4818
2.0,194,2055
3.0,27,330
4.0,4,41
5.0,0,9
20.0,8,76


In [16]:
# add prob
report_children_factors["prob"] = report_children_factors["sum"] / report_children_factors["count"]

In [17]:
report_children_factors

,sum,count,prob
children,,,
0.0,1063,14149,0.075129
1.0,444,4818,0.092154
2.0,194,2055,0.094404
3.0,27,330,0.081818
4.0,4,41,0.097561
5.0,0,9,0.000000
20.0,8,76,0.105263


### Conclusion

- Is there a relation between marital status and repaying a loan on time?

Those in a marriage and have defaulted are the majority in the dataset, while those who are not married have a lower rate of defaulting. This dispells the earlier hypothesis a co-dependant relationship may help in preventing a late loan payment. Currently, married couples are a high risk.

In [18]:
print(
    'Those that are in a marriage and have defaulted on a loan are {:.2%} of all clients'.format(married_debt_ratio)
)
print(
    'Those that are not married and have defaulted on a loan are {:.2%} of all clients'.format(not_married_debt_ratio)
)
print(
    'Those that are in a civil partnership and have defaulted on a loan are {:.2%} of all clients'.format(civil_married_debt_ratio)
)

Those that are in a marriage and have defaulted on a loan are 4.33% of all clients
Those that are not married and have defaulted on a loan are 1.27% of all clients
Those that are in a civil partnership and have defaulted on a loan are 1.80% of all clients


### Conclusion

- Is there a relation between income level and repaying a loan on time?

Based off the results, there is a negligible and dismissable difference between income levels for whether someone would default on a loan. Income level here provides no meaningful contribution for determining credit. It could be the case that most loan takers have a similar income level compared to the upper class, as it could be reasoned the upper class wouldn't have need for a loan in the first place.

In [19]:
print(
    'Those that are in the upper income bracket and have defaulted on a loan are {:.2%} of all of those that have defaulted on their loans'.format(over_income_debt_ratio)
)
print(
    'Those that are in the lower income bracket and have defaulted on a loan are {:.2%} of all of those that have defaulted on their loans'.format(under_income_debt_ratio)
)

Those that are in the upper income bracket and have defaulted on a loan are 3.56% of all of those that have defaulted on their loans
Those that are in the lower income bracket and have defaulted on a loan are 3.74% of all of those that have defaulted on their loans


### Conclusion

- How do different loan purposes affect on-time repayment of the loan?

Those that use the loan for a housing purchase are at the highest risk for not paying the loan on time, followed by car and education purchase closely tied behind, and the least risky being marriage. Housing loans may be the most risky as people may be more likely to take on a bigger loan for a nicer house than they can reasonably afford, possibily as a result of predatory real estate salespeople. Education may be lower because it is essentially able to pay for itself with the future career granted by it. Car purchases may also be less risky for a default compared to housing as cars are much cheaper than housing while also a necessity. Marriage is the safest as there are most likely two people working to pay off the loan rather than just one as it may be in the other categories.

In [20]:
print(
    'Those that use the loan for a housing purchase and have defaulted on a loan are {:.2%} of all clients'.format(purpose_housing_debt_ratio)
)
print(
    'Those that use the loan for marriage and have defaulted on a loan are {:.2%} of all clients'.format(purpose_marriage_debt_ratio)
)
print(
    'Those that use the loan for education and have defaulted on a loan are {:.2%} of all clients'.format(purpose_education_debt_ratio)
)
print(
    'Those that use the loan for a car purchase and have defaulted on a loan are {:.2%} of all clients'.format(purpose_cars_debt_ratio)
)

Those that use the loan for a housing purchase and have defaulted on a loan are 0.00% of all clients
Those that use the loan for marriage and have defaulted on a loan are 0.86% of all clients
Those that use the loan for education and have defaulted on a loan are 1.72% of all clients
Those that use the loan for a car purchase and have defaulted on a loan are 0.00% of all clients


### Conclusion

The overall data and conclusions here is the framework for determining the best credit report for determining the risk of a loan. Most of the results were unexpected, with some being expected with reasonable inferences that can be drawn. Most notably married individuals being the riskiest compared to those that aren't married is the most surprising, especially since it would initially be surmised that two people together with (probably) combined incomes would pay off loans much easier.

## General conclusion

Using the dataset and analyzing the data, a good idea for both a perfect and risky candidate for a loan can be determined. Overall, those with children is significantly safer of a risk for a loan than those who have none. This can be explained for the co-dependant relationship those with children likely have with the other parent, but this is disproven as someone who is married is also much more likely to default, whereas those with a civil partnership or those not married are almost twice as safe. This may lead to the possibility that unmarried couples with children are a safer loan, but more data may be needed to support that conclusion. According to the data income level makes little impact on the rate of late payment, with those under and above the median having a nearly indentical rate. Loan purpose may be the best determination for risk as those who use it for a housing purchase are much more risky, with those using it for a car purchase following behind in risk but almost twice as safe than a housing loan, with education and marriage being the safest purposes. 

The ideal person according to the data would be unmarried with children using the loan for marriage. This may be that those that are newly married are much more likely to stay together and rely on each other, both of which are likely to be contributing to paying back the loan as they both use it, and their co-dependance may also be a factor for ensuring the loan is sucessfully paid back on time. Individuals that are already married may also be co-dependant but the loan purpose may be more personal in nature rather than for the benefit of both, so it may only be the primary loan taker paying it instead of both and they may have progressed independance financially compared to newly weds, which may explain why already married couples are risky comparatively.